In [ ]:
import pandas as pd
import os # Using os.path, you can create and manipulate file and directory paths
import matplotlib.pyplot as plt

folder_path = 'c:\\Users\\magda\\Documents\\___DA_DATA\\PROJEKT_Czechitas_Honeywell\\csv_nove_soubory'
outputfolder = 'C:\\Users\\magda\\Documents\\___DA_DATA\\PROJEKT_Czechitas_Honeywell\\csv_new_outputs'


def dataprocessing(df, output_plot_path):

# The diff() method is used to compute the difference between values in consecutive rows (data points) within a single column. 
# It calculates the difference between the current value and the previous value for each row in the column. 
# Spoofing can be detected by a sudden and significant change in position or altitude. 
# Detection of such changes can be achieved by calculating the difference between timestamps in latitude, longitude, or altitude.

    df['delta_lat'] = df['g_lat'].diff()
    df['delta_lon'] = df['g_lon'].diff()
    df['delta_alt'] = df['g_alt_msl'].diff()
    
    # Threshold values for anomaly detection are set for latitude, longitude, and altitude.
    # abs represents the absolute value, ensuring negative numbers are converted to positive; we are interested in the magnitude of the change, disregarding its direction. The slash | means OR.
    # A change of 0.0005 degrees in latitude/longitude corresponds to approximately 55 meters on the Earth's surface.

    lat_threshold = 0.0005  
    lon_threshold = 0.0005  
    alt_threshold = 10      

    # Anomaly detection: returns the 'spoofing' column from the DataFrame df. 
    # This column contains True or False, indicating whether spoofing was detected for the respective row.

    df['spoofing'] = ((df['delta_lat'].abs() > lat_threshold) |
    (df['delta_lon'].abs() > lon_threshold) |
    (df['delta_alt'].abs() > alt_threshold))

    # Displays rows where a change (possible spoofing) was detected. Selects all rows from df where the value in the 'spoofing' column is True.
    # The result is a new DataFrame (spoofing_cases) that contains only the rows where spoofing was detected.

    spoofing_cases = df[df['spoofing']]

    # print(spoofing_cases)
    # Duration
    # Shifts the column using shift() by one position down.
    # df['prev_is_spoofing'] indicates the value in the previous row, whether spoofing was detected in the previous row (timestamp) or not.
    # It takes the value from the previous row and inverts it.

    df['prev_spoofing'] = df['spoofing'].shift(1, fill_value=False) # fill_value=False Fills the first row with the value False.

    # Identification of the start and end of spoofing:
    # Start of spoofing: This is determined by whether the current value is True and the previous value was False. 
    # The negation ~prev_spoofing == True means that the previous row was False and spoofing started in the current row.


    df['spoofing_start'] = (df['spoofing'] & ~df['prev_spoofing'])

    # End of spoofing:

    df['spoofing_end'] = (~df['spoofing'] & df['prev_spoofing'])

    # Time of the start of spoofing:

    start_times = df[df['spoofing_start']]['time']

    # Time of the end of spoofing:

    end_times = df[df['spoofing_end']]['time']

    # Duration of spoofing:

    spoofing_durations = end_times.values - start_times.values

    # Iterating over a list. The enumerate() function provides both the index of each element and the value of the element.
    # for i, duration in enumerate(spoofing_durations):
    #print(f'Spoofing {i+1} lasted: {duration:.2f} seconds')  # i+1 ensures numbering starts from 1, not 0. # .2f rounds to 2 decimal places

    # Total duration:

    total_duration = sum(spoofing_durations)

    # print(f'The total duration of all spoofing events is: {total_duration:.2f} seconds')

    import matplotlib.pyplot as plt

    # Creating a plot for latitude and longitude (drone's path)

    plt.plot(df['g_lon'], df['g_lat'], label='drone path')

    # Mark points with possible spoofing (red points)

    plt.scatter(df[df['spoofing']]['g_lon'], df[df['spoofing']]['g_lat'], color='red', label='Spoofing Detected')

    # Axis labels and legend
    
    plt.xlabel('Zeměpisná délka (rad)')
    plt.ylabel('Zeměpisná šířka (rad)')
    plt.legend()
    
    plt.savefig(output_plot_path, dpi=300)
    plt.close() 
    
    return df


In [ ]:

dataframes = []

# Iterating through all files in a folder. The os.listdir() function is part of the os module in Python and is used 
# to obtain a list of files and folders in a specified directory. This function returns all files and folders 
# found in the specified folder as a list of strings (file and folder names).

for csv_file in os.listdir(folder_path):

    # Checking if the file is a CSV. The endswith('.csv') function is a string method in Python that checks 
    # if the given string ends with a specific sequence of characters, in this case, whether the file name is in CSV format.

    if csv_file.endswith('.csv'):
        try:
            file_path = os.path.join(folder_path, csv_file) 
            output_file_path = os.path.join(outputfolder, csv_file)
            output_plot_path = os.path.join(outputfolder, os.path.splitext(csv_file)[0] + '_plot.png')
            df_old = pd.read_csv(file_path)
            df_edited = dataprocessing(df_old, output_plot_path)
            df_edited['filename'] = csv_file
            df_edited.to_csv(output_file_path, index=False)
            dataframes.append(df_edited)
        except Exception as e:
            print(f"Chyba při načítání souboru {file_path}: {e}")

            # Check for CSV and save the processed DataFrame to CSV

In [ ]:
len(dataframes) 

# The pd.concat() function combines all DataFrames in the list 'dataframes' into one large DataFrame.
# ignore_index=True: This argument ensures that the resulting DataFrame will have redefined (new) indices from 0 to n.
# The original indices of the individual DataFrames are ignored, and the DataFrames are concatenated as if they were continuous rows.

df_final = pd.concat(dataframes, ignore_index=True)

In [ ]:
df_final

In [ ]:
# Saving the DataFrame 'df' to a CSV file 'output.csv' without including the DataFrame indices (row numbers) in the saved CSV file.
# If index=False were not used, the indices would be saved as a separate column in the CSV file.

df_final.to_csv('finaldataset.csv', index=False)

In [ ]:
df_final.to_csv('finaldataset.csv', index=False)

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Loading the CSV file
df = pd.read_csv('finaldataset.csv')

# Converting columns with coordinates to geometry
geometry = [Point(xy) for xy in zip(df['g_lon'], df['g_lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Setting the coordinate reference system 
gdf.set_crs(epsg=4326, inplace=True)

# Displaying the map with visualization based on the 'value' column
gdf.plot(column='g_avg_cn0', legend=True)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Vizualizace průměrného CN0 (Carrier-to-Noise Ratio)')
plt.title('Vizualizace dat s využitím GeoPandas')
plt.show()

# The color scale on the right indicates the values of the column, representing noise. The color represents the signal strength (CN0).

In [ ]:
# This code displays a graph for each CSV file in the folder.
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import os

# Loading the CSV file
folder_path = 'c:\\Users\\magda\\Documents\\___DA_DATA\\PROJEKT_Czechitas_Honeywell\\csv_new_outputs'

# Iterating through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # Loading the CSV file
        df = pd.read_csv(file_path)
        
        # Creating geometry if columns with geographic data exist
        if 'g_lon' in df.columns and 'g_lat' in df.columns:
            geometry = [Point(xy) for xy in zip(df['g_lon'], df['g_lat'])]
            gdf = gpd.GeoDataFrame(df, geometry=geometry)
            gdf.set_crs(epsg=4326, inplace=True)
            
            # Plotting the graph
            gdf.plot(column='g_avg_cn0', legend=True)  # Použijte správný sloupec
            plt.title(f'Vizualizace dat z souboru: {filename}')
            plt.xlabel('Longitude')
            plt.ylabel('Latitude')
            plt.show()